In [1]:
import struct
import pyodbc
from tabulate import tabulate
from sqlalchemy import create_engine
import pandas as pd
from azure.identity import AzureCliCredential

# Parameters
server = "jarbdatabase.database.windows.net"
database = "Azure_EmployeeDB"

# Acquire token
credential = AzureCliCredential()
token = credential.get_token("https://database.windows.net/.default")

# Convert token to ODBC struct
token_bytes = token.token.encode("utf-8")
exptoken = b"".join(bytes([b]) + b"\0" for b in token_bytes)
token_struct = struct.pack("=i", len(exptoken)) + exptoken

# Build connection string
conn_str = f"Driver={{ODBC Driver 18 for SQL Server}};Server={server};Database={database};TrustServerCertificate=Yes;"

# Create SQLAlchemy engine using pyodbc + token
engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={conn_str}",
    connect_args={"attrs_before": {1256: token_struct}}
)

#Query to execute
query = "SELECT TOP 10 * FROM [SalesLT].[Customer]"
df = pd.read_sql(query, engine)
df_sql = df[df["CustomerID"] == 1]
print(tabulate(df_sql, headers = 'keys', tablefmt = 'pretty'))


+---+------------+-----------+-------+-----------+------------+----------+--------+--------------+-------------------------+------------------------------+--------------+----------------------------------------------+--------------+--------------------------------------+---------------------+
|   | CustomerID | NameStyle | Title | FirstName | MiddleName | LastName | Suffix | CompanyName  |       SalesPerson       |         EmailAddress         |    Phone     |                 PasswordHash                 | PasswordSalt |               rowguid                |    ModifiedDate     |
+---+------------+-----------+-------+-----------+------------+----------+--------+--------------+-------------------------+------------------------------+--------------+----------------------------------------------+--------------+--------------------------------------+---------------------+
| 0 |     1      |   False   |  Mr.  |  Orlando  |     N.     |   Gee    |        | A Bike Store | adventure-works\pam